# Super Hero



In [1]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
## Importing the OS and JSON Modules
import os,json

## Superhero Info

### Load Dataset

In [2]:
info = pd.read_csv('Data/superhero_info - superhero_info.csv')
info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


### Clean Dataset


In [3]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
info.duplicated().sum()

0

In [5]:
info.nunique()

Hero|Publisher    457
Gender              3
Race               52
Alignment           4
Hair color         29
Eye color          21
Skin color         14
Measurements      287
dtype: int64

### Convert dictionary column into multiple columns

#### Convert entire Measurements column using json.loads

In [6]:
test_measurement = info.loc[1,'Measurements']
test_measurement

"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"

In [7]:
type(test_measurement)

str

In [8]:
# Replace single quotes with double quotes
test_measurement = test_measurement.replace("'", '"')
test_measurement

'{"Height": "191.0 cm", "Weight": "65.0 kg"}'

In [9]:
json.loads(test_measurement)

{'Height': '191.0 cm', 'Weight': '65.0 kg'}

In [10]:
type(json.loads(test_measurement))

dict

In [11]:
# Replace '' with ""
info['Measurements'] = info['Measurements'].str.replace("'",'"')

#Apply json.loads
info['Measurements'] = info['Measurements'].apply(json.loads)

In [12]:
test_measurement = info.loc[7, 'Measurements']
test_measurement

{'Height': '191.0 cm', 'Weight': '104.0 kg'}

In [13]:
type(test_measurement)

dict

In [14]:
#Use .apply pd.Series to convert a dict to columns
info['Measurements'].apply(pd.Series)

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [15]:
#use pd.concat to concatenate the new columns 
info = pd.concat([info, info['Measurements'].apply(pd.Series)], axis =1)
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Height,Weight
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",203.0 cm,441.0 kg
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",191.0 cm,65.0 kg


In [16]:
#drop the original column- Measurements
info = info.drop(columns = 'Measurements')
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height,Weight
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,203.0 cm,441.0 kg
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0 cm,65.0 kg


#### Split 'Hero|Publisher' to different columns

In [17]:
info['Hero|Publisher'].head()

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
2              Abin Sur|DC Comics
3       Abomination|Marvel Comics
4     Absorbing Man|Marvel Comics
Name: Hero|Publisher, dtype: object

In [18]:
test_heropub = info.loc[1, 'Hero|Publisher']
test_heropub

'Abe Sapien|Dark Horse Comics'

In [19]:
info['Hero'] = info['Hero|Publisher'].apply(lambda x: x.split('|')[0])
info['Publisher'] = info['Hero|Publisher'].apply(lambda x: x.split('|')[1])


In [20]:
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height,Weight,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,203.0 cm,441.0 kg,A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0 cm,65.0 kg,Abe Sapien,Dark Horse Comics


In [21]:
#drop the original column- Hero|Publisher
info = info.drop(columns = 'Hero|Publisher')
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height,Weight,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,203.0 cm,441.0 kg,A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0 cm,65.0 kg,Abe Sapien,Dark Horse Comics


In [22]:
info = info.reindex(columns=['Hero', 'Publisher'] + list(info.columns[:-2]))

In [23]:
info.head(2)

,Hero,Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height,Weight
0,A-Bomb,Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,203.0 cm,441.0 kg
1,Abe Sapien,Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0 cm,65.0 kg


In [24]:
# save list of categorical column name.
categorical_col = info.select_dtypes('object').columns
categorical_col

Index(['Hero', 'Publisher', 'Gender', 'Race', 'Alignment', 'Hair color',
       'Eye color', 'Skin color', 'Height', 'Weight'],
      dtype='object')

In [25]:
# Check for nunique for categorical columns
for col in categorical_col:
  print(f'Value Counts for {col}')
  print(info[col].value_counts())
  print('\n')

Value Counts for Hero
Spider-Man        3
Black Canary      2
Nova              2
Captain Marvel    2
Toxin             2
                 ..
Feral             1
Falcon            1
Fabian Cortez     1
Exodus            1
Zoom              1
Name: Hero, Length: 455, dtype: int64


Value Counts for Publisher
Marvel Comics        297
DC Comics            138
Dark Horse Comics     11
George Lucas           5
Team Epic TV           4
Shueisha               4
Star Trek              2
Unknown                1
Image Comics           1
Name: Publisher, dtype: int64


Value Counts for Gender
Male       321
Female     134
Unknown      8
Name: Gender, dtype: int64


Value Counts for Race
Human                156
Unknown              145
Mutant                51
Human / Radiation     11
Android                7
Symbiote               7
Alien                  6
Demon                  6
Cyborg                 6
Kryptonian             5
Atlantean              5
Asgardian              5
Inhuman       

#### Convert 'Height' and 'Weight' columns to numeric type

In [26]:
info['Height'] = info['Height'].str.replace(" cm", "")

In [27]:
info['Height'] = info['Height'].astype(float)

In [28]:
info['Weight'] = info['Weight'].str.replace(" kg", "")

In [29]:
info['Weight'] = info['Weight'].astype(float)

In [30]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Hero        463 non-null    object 
 1   Publisher   463 non-null    object 
 2   Gender      463 non-null    object 
 3   Race        463 non-null    object 
 4   Alignment   463 non-null    object 
 5   Hair color  463 non-null    object 
 6   Eye color   463 non-null    object 
 7   Skin color  463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


## Superhero Powers

### Load Dataset

In [31]:
powers = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


### Clean Dataset


In [32]:
powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [33]:
powers['Powers'].head()

0           Agility,Super Strength,Stamina,Super Speed
1    Accelerated Healing,Durability,Longevity,Super...
2    Agility,Accelerated Healing,Cold Resistance,Du...
3                                   Lantern Power Ring
4    Accelerated Healing,Intelligence,Super Strengt...
Name: Powers, dtype: object

In [34]:
powers.rename(columns={'hero_names': 'Hero'}, inplace=True)

### Convert Power column of lists into one-hot-encoded columns

In [35]:
#Convert str to list by replacing '' to ""
powers['powers_split'] = powers['Powers'].str.split(',')


In [36]:
powers['powers_split'].apply(pd.Series)
powers['powers_split'].head()

0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: powers_split, dtype: object

In [37]:
powers.head()

,Hero,Powers,powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring]
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre..."


In [38]:
# using .explode
powers_exploded = powers.explode('powers_split')
powers_exploded[['Hero','Powers','powers_split']].head(2)

,Hero,Powers,powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength


In [39]:
## saving the unique values from the exploded column
cols_to_make = powers_exploded['powers_split'].dropna().unique()

for col in cols_to_make:
    powers[col] = powers['Powers'].str.contains(col)

/var/folders/5g/dfbryjbs45z6hwl2nsnbw_ch0000gn/T/ipykernel_13699/1349782695.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col] = powers['Powers'].str.contains(col)
/var/folders/5g/dfbryjbs45z6hwl2nsnbw_ch0000gn/T/ipykernel_13699/1349782695.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col] = powers['Powers'].str.contains(col)
/var/folders/5g/dfbryjbs45z6hwl2nsnbw_ch0000gn/T/ipykernel_13699/1349782695.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.

In [40]:
powers.head()

,Hero,Powers,powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
#Drop 'power_split' and 'Powers' columns
powers = powers.drop(columns=['powers_split', 'Powers'])
powers.head()

,Hero,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Merging Info and Powers to one dataset: Superhero

In [42]:
superhero = pd.merge(info, powers, on = 'Hero', how='inner')
superhero.head(10)

,Hero,Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,A-Bomb,Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Abin Sur,DC Comics,Male,Ungaran,good,No Hair,blue,red,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Abomination,Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Absorbing Man,Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False
5,Adam Strange,DC Comics,Male,Human,good,Blond,blue,Unknown,185.0,88.0,...,False,False,False,False,False,False,False,False,False,False
6,Agent Bob,Marvel Comics,Male,Human,good,Brown,brown,Unknown,178.0,81.0,...,False,False,False,False,False,False,False,False,False,False
7,Agent Zero,Marvel Comics,Male,Unknown,good,Unknown,Unknown,Unknown,191.0,104.0,...,False,False,False,False,False,False,False,False,False,False
8,Air-Walker,Marvel Comics,Male,Unknown,bad,White,blue,Unknown,188.0,108.0,...,False,False,False,False,False,False,False,False,False,False
9,Ajax,Marvel Comics,Male,Cyborg,bad,Black,brown,Unknown,193.0,90.0,...,False,False,False,False,False,False,False,False,False,False


## Saving Merged Dataset to Data Folder

In [43]:
#Saving merged df superhero to Data folder
superhero.to_csv('Data/superhero_merged.csv', index=False)

## Questions

1. Compare the average weight of super powers who have Super Speed to those who do not.

In [44]:
superhero['Weight'].mean()

113.58963282937366

In [45]:
sspeed_heros = superhero[superhero['Super Speed'] == True]
no_sspeed_heros = superhero[superhero['Super Speed'] == False]

In [49]:
avg_sspeed_weight = sspeed_heros['Weight'].mean().round()
avg_no_sspeed_weight = no_sspeed_heros['Weight'].mean().round()

In [50]:
print(f'The average weight of Heros with the Super Speed power is {avg_sspeed_weight}.')
print(f'The average weight of Heros without the Super Speed power is {avg_no_sspeed_weight}.')

The average weight of Heros with the Super Speed power is 129.0.
The average weight of Heros without the Super Speed power is 102.0.


2. What is the average height of heroes for each publisher?

In [51]:
avg_height_per_publisher = superhero.groupby('Publisher')['Height'].mean().round()
avg_height_per_publisher

Publisher
DC Comics            182.0
Dark Horse Comics    177.0
George Lucas         160.0
Image Comics         211.0
Marvel Comics        192.0
Shueisha             172.0
Star Trek            182.0
Team Epic TV         181.0
Unknown              178.0
Name: Height, dtype: float64